In [71]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options ### CONFIGURADO CON EDGE
import pandas as pd
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
#import cv2
#import pytesseract
import base64
import numpy as np
import os
import random
import re

In [81]:
options = Options()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_argument("--disable-notifications")

def opendriver(options):
    driver = webdriver.Chrome(options=options)
    time.sleep(2)
    driver.maximize_window()
    print("Abriendo WebDriver")
    time.sleep(2)
    return driver

driver = opendriver(options)

Abriendo WebDriver


In [26]:
busqueda = "Coquimbo Region, Chile"
def apertura_marketplace(driver, busqueda):
    url = "https://www.booking.com/searchresults.es.html?aid=304142&label=gen173nr-1FCAEoggI46AdIM1gEaC-IAQGYAQq4ARfIAQzYAQHoAQH4AQ2IAgGoAgO4AvaLoJ8GwAIB0gIkZTdlMzVlZWItNjFlYi00MGI3LThiOTUtYzI3NTY2YTZmNzEz2AIG4AIB&sid=f66f439f510c490173e2f538c8f7dfc0&sb=1&sb_lp=1&src=region&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fregion%2Fcl%2Fcoquimbo.es.html%3Faid%3D304142%26label%3Dgen173nr-1FCAEoggI46AdIM1gEaC-IAQGYAQq4ARfIAQzYAQHoAQH4AQ2IAgGoAgO4AvaLoJ8GwAIB0gIkZTdlMzVlZWItNjFlYi00MGI3LThiOTUtYzI3NTY2YTZmNzEz2AIG4AIB%26sid%3Df66f439f510c490173e2f538c8f7dfc0%26&ss=Coquimbo+Region&is_ski_area=0&ssne=Coquimbo+Region&ssne_untouched=Coquimbo+Region&region=1352&checkin_year=&checkin_month=&checkout_year=&checkout_month=&efdco=1&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1"
    driver.get(url)
    time.sleep(3)
    #print("Cargando Booking y realizando búsqueda de '", busqueda, "'")
    #buscador = driver.find_element(By.XPATH,'//input[@placeholder="¿Adónde vas?"]')
    #buscador.click()
    #buscador.send_keys(Keys.CONTROL, "a")
    #buscador.send_keys(Keys.DELETE)
    #time.sleep(1)
    #buscador = driver.find_element(By.XPATH, '//input[@id="bigsearch-query-location-input"]')
    #buscador.clear()   ]
    #buscador.send_keys(busqueda)
    #button = driver.find_element(By.XPATH,'//button[@type="submit"]')
    #button.click()
    #time.sleep(3)
apertura_marketplace(driver, busqueda)

In [85]:

link =  driver.current_url
df = pd.DataFrame()
def scrap(driver, link, df):
    driver.get(link)
    time.sleep(3)
    
    ## Scrolldown
    #googlemap = driver.find_element(By.XPATH, '//h2[contains(text(), "A dónde irás")]')
    #driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", googlemap)
    
    time.sleep(1)
    try: 
        datebut = driver.find_element(By.XPATH, '//*[@id="basiclayout"]/div[1]/div[2]/div/div/div[1]/div[1]/div[1]/div[2]/a')
        datebut.click()
    except: 
        print("")
        
    Link = {"Link":link}
    
    direccion = driver.find_element(By.XPATH, '//span[contains(@class, "address_subtitle")]').text
    Direccion = {"Direccion": direccion}
    print(Direccion)

    pattern = re.compile(r"\d+")
    # Replace all numeric values in the text with an empty string:
    text_without_numbers = re.sub(pattern, "", direccion.split(",")[1])
    Comuna = {"Comuna": text_without_numbers.strip()}    
    print(Comuna)
        
    #<<<Baños = {"Baños": driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/ol/li[4]/span[2]').text}

    datos_seccion = driver.find_element(By.XPATH, '//*[@id="maxotelRoomArea"]/section')
    #print(datos_seccion.text)
    text = datos_seccion.text

    # Use the following pattern to match the required information:
    # (\d+) (cama|camas|litera|literas)
    pattern = re.compile(r"(\d+) (cama individual|camas individuales|cama doble|camas dobles|litera|literas|sofá cama)")
    # Find all matches in the text:
    matches = re.findall(pattern, text)
    camas = []
    bed_count = 0
    # Print the matches:
    for match in matches:
        camas.append(match[0]+" "+match[1])
        #print("MATCH ", match)
        # Check if the bed type is a double bed
        if match[1] in ['cama doble', 'camas dobles', 'litera', 'literas']:
            bed_count += 2 * int(match[0])
        # Check if the bed type is a single bed
        elif match[1] in ['cama individual', 'camas individuales', 'futón']:
            bed_count += int(match[0])
        # Check if the bed type is a sofa bed
        elif match[1] == 'sofá cama':
            bed_count += int(match[0])
    #print("texto extraído", camas)
    print("bedcount", bed_count)
    

    Camas = {"Camas": bed_count}
    Capacidad = {"Capacidad": bed_count}
    
    divshabit = driver.find_elements(By.XPATH, '//*[@id="maxotelRoomArea"]/section/div/div[1]')
    print(divshabit)
    print("items divshabit", len(divshabit))
    infohabits = {}
    for n in range(1,int(len(divshabit)-1),1):
        #print("for", n)
        key = divshabit[n].find_element(By.XPATH, './div[1]').text
        content = divshabit[n].find_element(By.XPATH, './div[2]').text
        dictemp = {key:content}
        infohabits.update(dictemp)
    print(infohabits)
    info_habitaciones = {"Info_Habitaciones":infohabits}
    ### >> Tipo            
    
    infohabitaciones = driver.find_elements(By.XPATH, '//*[@id="maxotelRoomArea"]/section/div')
    print(len(infohabitaciones)-1)
    infohabitaciones2 = driver.find_elements(By.XPATH, '//*[@id="maxotelRoomArea"]/section/div/div[1]/div[2]/div')
    nnohabt = 0
    for item in infohabitaciones2:
        pattern = re.compile(r"(y)")
        matches = re.findall(pattern, item.text)
        nnohabt += len(matches)*2

    if len(infohabitaciones)-1 == 1:
        infohabitaciones = driver.find_element(By.XPATH, '//*[@id="maxotelRoomArea"]/section/div[2]/div[1]/div[2]').text
        print(infohabitaciones)
        pattern = re.compile(r"(Dormitorio)")
        matches = re.findall(pattern, infohabitaciones)
        #print(len(matches))
        Habitaciones = {"Habitaciones": len(matches)}
    elif len(driver.find_elements(By.XPATH, '//*[@id="maxotelRoomArea"]/section/div/div[1]/div[2]/div'))-nnohabt == len(infohabitaciones)-1:
        Habitaciones = {"Habitaciones": str(len(infohabitaciones)-1)+"+"}
    else:
        pattern = re.compile(r"(Dormitorio)")
        nhabt = 0
        for item in driver.find_elements(By.XPATH, '//*[@id="maxotelRoomArea"]/section/div/div[1]/div[2]'):
            print("space")
            print(item.text)
            matches = re.findall(pattern, item.text)
            if len(matches) == 0: nhabt += 1
            else: nhabt += len(matches)
            print(matches)
            print(nhabt)
        Habitaciones = {"Habitaciones": str(nhabt)+"+"}

    Camas = {"Camas": bed_count}
    Capacidad = {"Capacidad": bed_count}

    if "+" in str(Habitaciones["Habitaciones"]):
        Camas = {"Camas": str(bed_count)+"+"}
        Capacidad = {"Capacidad": str(bed_count)+"+"}

    print(Camas)
    print(Capacidad)
    print(Habitaciones)
    
    #Precio = {"Precio":driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[1]/div/span/div/span[1]').text}
    
    try: Nota = {"Nota":driver.find_element(By.XPATH, '//*[@id="guest-featured_reviews__horizontal-block"]/div[2]/div[1]/div/div[2]/div/button/div/div/div[1]').text.replace(",",".")}
    except: Nota = {"Nota": None}
    print(Nota)
    #try: Nnota = {"Nnota":driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[2]/span/span[3]/button/span').text}
    #except: Nnota = {"Nota": None}
    
    Descripción = {"Descripción":driver.find_element(By.XPATH, '//*[@id="property_description_content"]').text}
    print(Descripción)

    Titulo1 = {"Titulo1":driver.find_element(By.XPATH, '//*[@id="hp_hotel_name"]/div/div/h2').text}
    print(Titulo1)
    
    #Titulo2 = {"Titulo2":driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/div/h2').text}
    
    Servicios = {"Servicios":driver.find_element(By.XPATH, '//*[@id="hp_facilities_box"]').text}
    print(Servicios)
    
    try: Anfitrion = {"Anfitrion":driver.find_element(By.XPATH, '//div[@data-testid="host-image"]/../div[2]').text}
    except: Anfitrion = {"Anfitrion": None}
    try: Descripcionanfi = {"Descripcionanfi":driver.find_element(By.XPATH, '//div[@data-testid="host-image"]/../../div[2]').text}
    except: Descripcionanfi = {"Descripcionanfi": None}


    

    try: text = driver.find_element(By.XPATH, '//p[@class="summary  hotel_meta_style"]').text
    except: text = driver.find_element(By.XPATH, '//p[@class="summary hotel_meta_style"]').text
    print("DEBUG", text)
    pattern = re.compile(r"(\d+) (\w+) (\d+)")
    # Find all matches in the text:
    matches = re.findall(pattern, text)
    if matches:
        # Get the first match (there should only be one match):
        try:   
            match = matches[0]
            day, month_str, year = match
            months = ["ene", "feb", "mar", "abr", "may", "jun", "jul", "ago", "sep", "oct", "nov", "dic"]
            # Convert the month from string to number:
            month = months.index(month_str.lower()) + 1
            # Format the date as desired:
            date = f"{day}/{month}/{year}"
        except: 
            match = matches[1]
            day, month_str, year = match
            months = ["ene", "feb", "mar", "abr", "may", "jun", "jul", "ago", "sep", "oct", "nov", "dic"]
            # Convert the month from string to number:
            month = months.index(month_str.lower()) + 1
            # Format the date as desired:
            date = f"{day}/{month}/{year}"
    
    Registro = {"Registro":date}
    print(Registro)
    
    #try: Ubides = {"Ubides":driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[5]/div/div/div/div[2]/div/section/div[4]/div/div/div/div[2]/span/span').text}
    #except: Ubides = {"Ubides":None}
    ubicc = driver.find_element(By.XPATH, '//a[@id="hotel_sidebar_static_map"]').get_attribute("data-atlas-latlng")
    Ubicacion = {"Ubicacion": ubicc}
    print(Ubicacion)
    lat = {"lat": ubicc.split(",")[0]}
    print(lat)
    long = {"long": ubicc.split(",")[1]}
    print(long)
    dicts = [Link, Direccion, Camas, Capacidad, Habitaciones, Nota,
        Descripción, Titulo1, Servicios, Anfitrion, Descripcionanfi, Registro, Ubicacion, lat, long, info_habitaciones]
    #print(dicts)
    for dic in dicts:
        Comuna.update(dic)

    dfmin = pd.DataFrame([Comuna])
    df = pd.concat([df,dfmin])
    return df

dfmin = scrap(driver, link, df)

{'Direccion': 'Vicuña 415, 1770000 La Serena, Chile'}
{'Comuna': 'La Serena'}
bedcount 16
[<selenium.webdriver.remote.webelement.WebElement (session="504495f240ae08ba09070d290b5c9864", element="791ca0db-0ac7-4d62-b212-327eea47231c")>, <selenium.webdriver.remote.webelement.WebElement (session="504495f240ae08ba09070d290b5c9864", element="5766c801-e031-474e-b209-24076b7d00d3")>, <selenium.webdriver.remote.webelement.WebElement (session="504495f240ae08ba09070d290b5c9864", element="6dbafbcf-4fc7-4199-9654-e664bfc52266")>, <selenium.webdriver.remote.webelement.WebElement (session="504495f240ae08ba09070d290b5c9864", element="f357e6a9-6774-4a34-83ec-e7a9921906e9")>, <selenium.webdriver.remote.webelement.WebElement (session="504495f240ae08ba09070d290b5c9864", element="6b468296-fc72-4192-aa8c-860cff92241d")>, <selenium.webdriver.remote.webelement.WebElement (session="504495f240ae08ba09070d290b5c9864", element="dfc0d61c-a1f7-4c4a-820e-a427dba85dc5")>, <selenium.webdriver.remote.webelement.WebElem

In [57]:
dfmin["Info_Habitaciones"]

0    {'Habitación Individual Estándar': '1 cama ind...
Name: Info_Habitaciones, dtype: object

In [45]:
df = pd.DataFrame()
def scrap_links(driver, df):
    initial_url = driver.current_url
    #print(type(initial_url))
    #print(initial_url)
    items = driver.find_elements(By.XPATH, '//a[@data-testid="title-link"]')
    print(len(items))
    links = []
    for item in items:
        #print(item.get_attribute("href"))
        links.append(item.get_attribute("href"))

    for link in links:
        try: df = scrap(driver, link, df)
        except: 
            print("POP UP ACTIVO")
            time.sleep(10)
            #cerrar_button = driver.find_element(By.XPATH, "//button[@aria-label='Cerrar']")
            #cerrar_button.click()
            #time.sleep(3)
            df = scrap(driver, link, df)

    driver.get(initial_url)

    return df

df = scrap_links(driver, df)

26
{'Direccion': 'Uno Oriente, 1940000 Los Vilos, Chile'}
{'Comuna': 'Los Vilos'}
bedcount 8
[<selenium.webdriver.remote.webelement.WebElement (session="80b3934960312b82d8338a724388f9c7", element="b1430e35-8389-477a-857a-eba32d601a0c")>, <selenium.webdriver.remote.webelement.WebElement (session="80b3934960312b82d8338a724388f9c7", element="be0b7fe1-e8f4-4de5-8892-c33da0e8528c")>]
2
1
1
1
1
Dormitorio 1:1 cama individual
Dormitorio 2:1 cama individualy2 literas
Dormitorio 3:1 cama doble
{'Camas': 8}
{'Capacidad': 8}
{'Habitaciones': 3}
{'Nota': '9.0'}
{'Descripción': 'Las Cabañas Mamut uno oriente 146 se encuentran en Los Vilos, en la región de Coquimbo, y tienen balcón. Se encuentra frente a la playa y ofrece terraza, aparcamiento privado gratuito y WiFi gratuita.\nLa casa consta de 3 dormitorios, cocina y 2 baños. La casa también incluye 2 baños.\nLa playa de Los Vilos está a 500 metros de la casa. El aeropuerto más cercano es el Club Aéreo San Felipe, ubicado a 172 km de las Cabañas M

In [46]:
dfx = df.reset_index()

In [51]:
print(dfx["Info_Habitaciones"][21])
dfx.tail(10)

{'Habitación Individual Estándar': '1 cama individual', 'Habitación Doble Superior': '2 camas individuales'}


,index,Comuna,Link,Direccion,Camas,Capacidad,Habitaciones,Nota,Descripción,Titulo1,Servicios,Anfitrion,Descripcionanfi,Registro,Ubicacion,lat,long,Info_Habitaciones
16,0,La Serena,https://www.booking.com/hotel/cl/canto-del-mar...,"Avenida del Mar 2200, 1710381 La Serena, Chile",17+,17+,9+,7.7,¡Puedes conseguir un descuento Genius en Hotel...,Hotel Canto del Mar,Servicios de Hotel Canto del Mar\nVer disponib...,None,None,4/8/2011,"-29.91630009,-71.27511531",-29.91630009,-71.27511531,{'Habitación con cama grande y vistas al mar':...
17,0,km .,https://www.booking.com/hotel/cl/cabanas-el-al...,"Camino Principal Cochiguaz, km 5.5, 1770000 El...",9+,9+,5+,9.5,¡Puedes conseguir un descuento Genius en Cabañ...,Cabañas El Albaricoque Cochiguaz,Servicios de Cabañas El Albaricoque Cochiguaz\...,None,None,26/5/2017,"-30.12252584,-70.44259569",-30.12252584,-70.44259569,{'Chalet Superior': 'Dormitorio :1 cama doble ...
18,0,Los Vilos,https://www.booking.com/hotel/cl/cabana-en-amb...,"1 Oriente, 1940306 Los Vilos, Chile",4,4,3,8.7,Cabaña en ambiente familiar is situated in Los...,Cabaña en ambiente familiar,Servicios de Cabaña en ambiente familiar\nVer ...,None,None,31/1/2023,"-31.91045365,-71.50017550",-31.91045365,-71.50017550,{'Apartamento de 3 dormitorios': 'Dormitorio 1...
19,0,PARCELA LOTE C,https://www.booking.com/hotel/cl/qamawi.es.htm...,"KM 22, 45 PARCELA 38 LOTE C, 1700000 Pan de Az...",12+,12+,5+,None,El Qamawi Elqui se encuentra en Pan de Azúcar ...,Qamawi Cabañas Loft Spa Valle Elqui,Servicios de Qamawi Cabañas Loft Spa Valle Elq...,None,None,4/9/2017,"-29.99399200,-71.05697900",-29.99399200,-71.05697900,{'Apartamento': 'Dormitorio 1:2 literas Dormit...
20,0,Pisco Elqui,https://www.booking.com/hotel/cl/el-milagro.es...,"Arturo Prat Esq Baquedano, 1700000 Pisco Elqui...",11+,11+,5+,8.8,¡Puedes conseguir un descuento Genius en Hotel...,Hotel El Milagro,Servicios de Hotel El Milagro\nVer disponibili...,None,None,8/5/2013,"-30.12204311,-70.49324870",-30.12204311,-70.49324870,"{'Habitación Doble': '1 cama doble', 'Suite co..."
21,0,Ovalle,https://www.booking.com/hotel/cl/gran-ovalle.e...,"Benjamin Vicuña Mackenna 210, 1840000 Ovalle, ...",21+,21+,9+,8.5,El Gran Hotel Ovalle ofrece alojamiento en Ova...,Gran Hotel Ovalle,Servicios de Gran Hotel Ovalle\nVer disponibil...,None,None,12/7/2018,"-30.60302500,-71.20059300",-30.60302500,-71.20059300,{'Habitación Individual Estándar': '1 cama ind...
22,0,La Serena,https://www.booking.com/hotel/cl/serena-dream....,"Finlandia 881, 1700000 La Serena, Chile",10+,10+,4+,8.7,¡Puedes conseguir un descuento Genius en Apart...,Aparthotel Serena Dream,Servicios de Aparthotel Serena Dream\nVer disp...,None,None,10/12/2010,"-29.91263495,-71.27012104",-29.91263495,-71.27012104,{'Apartamento con vistas a la piscina': 'Dormi...
23,0,Punta de Choros,https://www.booking.com/hotel/cl/casa-quelita....,"Avenida Los de Aguirre s/n punta de choros, 17...",11,11,2,7.9,La Casa Quelita se encuentra en Punta de Choro...,Casa Quelita,Servicios de Casa Quelita\nVer disponibilidad\...,ANA ROjAS,"El alojamiento es bastante amplio tranquilo, a...",22/10/2022,"-29.24409740,-71.46036260",-29.24409740,-71.46036260,{'Casa de 2 dormitorios': 'Dormitorio 1:3 cama...
24,0,Tongoy,https://www.booking.com/hotel/cl/cabana-en-ton...,"D-510 79, 1780000 Tongoy, Chile",4,4,2,9.5,La Cabaña en Tongoy es un establecimiento con ...,Cabaña en Tongoy,Servicios de Cabaña en Tongoy\nVer disponibili...,Elvira,Tenemos un lugar tranquilo en un sector de par...,11/1/2022,"-30.28258638,-71.48929525",-30.28258638,-71.48929525,{'Apartamento de 2 dormitorios': 'Dormitorio 1...
25,0,Alcoguaz,https://www.booking.com/hotel/cl/casas-amancay...,"Camino a Alcohuaz sin numero, Alcoguaz, Chile",8+,8+,3+,9.2,"Las Casas Amancay se encuentran en Alcoguaz, e...",Casas Amancay,Servicios de Casas Amancay\nVer disponibilidad...,Veronica,"ALCOHUAZ. Exclusiva propiedad de tres casas, a...",18/8/2017,"-30.22364693,-70.49391389",-30.22364693,-70.49391389,{'Casa': 'Dormit

In [70]:
def siguiente_pagina(driver):
    button = driver.find_element(By.XPATH, '//button[@aria-label="Página siguiente"]')
    print("Cargando siguiente página")
    #time.sleep(5)
    #linkbut = button.get_attribute("href")
    #print("Link CARGANDO", button.get_attribute("href"))
    if button.is_enabled():
        button.click()
    else: 
        print("NO HAY SIGUIENTE PÁGINA")
        y = 1
        return y
siguiente_pagina(driver)

Cargando siguiente página
NO HAY SIGUIENTE PÁGINA
